In [4]:
from pathlib import Path

from llama_index.core import Document, VectorStoreIndex
from llama_index.core.node_parser import SimpleFileNodeParser
from llama_index.readers.file.epub.base import EpubReader


ollama-Embedding

In [1]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

pass_embedding = ollama_embedding.get_text_embedding_batch(
    ["This is a passage!", "This is another passage"], show_progress=True
)
print(pass_embedding)

query_embedding = ollama_embedding.get_query_embedding("Where is blue?")
print(query_embedding)

Generating embeddings:   0%|          | 0/2 [00:00<?, ?it/s]

[[0.3456532955169678, 0.4191662073135376, -0.8692266345024109, -0.3679901957511902, 0.4809704124927521, 1.1170220375061035, -0.3527449667453766, 0.2603655457496643, -0.21593888103961945, 0.7574427723884583, -0.04337773844599724, -0.3200320601463318, 0.02412106841802597, -0.042592331767082214, -1.143682599067688, 0.6347431540489197, 0.9669202566146851, -0.11717496812343597, -0.16181741654872894, -0.4683489203453064, 0.25014176964759827, -0.5403469204902649, -0.047669559717178345, 0.35906746983528137, -0.0886995792388916, 0.2360984832048416, -0.7830517292022705, 0.6834762692451477, 0.6642807722091675, 0.3661815822124481, 0.2029114067554474, 0.5619815587997437, -0.5312076807022095, 0.11286177486181259, -0.10736924409866333, 0.396139919757843, -1.0454492568969727, -0.6157737970352173, 0.33350586891174316, -0.17501765489578247, -0.026572421193122864, 0.2542966306209564, 0.7814412117004395, -0.4957478940486908, 0.4523408114910126, 0.37871354818344116, -0.5471406579017639, 1.0484095811843872,

In [4]:
test_data_path = Path("../data/")
# book_name，组合成新路径
book_name = "非暴力沟通.epub"
book_path = test_data_path / book_name
docs = EpubReader().load_data(book_path)

parser = SimpleFileNodeParser()
book_nodes = parser.get_nodes_from_documents(docs)

/Users/pegasus/Library/Caches/pypoetry/virtualenvs/readai-vv5wOT8E-py3.10/lib/python3.10/site-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


In [ ]:
# 检查节点数量，输出前10个节点
print(f"节点数量: {len(book_nodes)}")
print("前10个节点:")
for i, node in enumerate(book_nodes[:10]):
    print(f"节点 {i + 1}:")
    print(f"文本内容: {node.text}")
    print(f"元数据: {node.metadata}")
    print("节点内容长度: ", len(node.text))
    print("-" * 50)

优化节点分割，在epubreader基础上，能否结合章节内容分割

In [6]:
from langchain_community.document_loaders import UnstructuredEPubLoader
import json

# 第一步：使用 UnstructuredEPubLoader 加载 EPUB 文档
test_data_path = Path("../data/")
# book_name，组合成新路径
book_name = "非暴力沟通.epub"
book_path = test_data_path / book_name
loader = UnstructuredEPubLoader(book_path, mode="elements")
documents = loader.load()

# 第二步：转换为 LlamaIndex 的 Document 对象
llama_documents = [
    Document(text=doc.page_content, metadata=doc.metadata) for doc in documents
]

parser = SimpleFileNodeParser()
book_nodes = parser.get_nodes_from_documents(llama_documents)
print(f"节点数量: {len(book_nodes)}")
print("前10个节点:")

nodes_data = []
# 将每个节点信息整理为json_dict输出到文件中
for node in book_nodes:
    node_info = {
        "text": node.text,
        "metadata": node.metadata,
        "text_length": len(node.text),
    }
    nodes_data.append(node_info)
# 将节点信息列表写入到 JSON 文件
output_file = "nodes_data.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(nodes_data, f, ensure_ascii=False, indent=4)

print(f"节点信息已成功保存到 {output_file}")

# 第三步：构建索引（注意类名变更）
# index = VectorStoreIndex.from_documents(llama_documents)

# # 第四步：查询索引
# query_engine = index.as_query_engine()
# response = query_engine.query("请输入您的查询内容")
# print(response)

节点数量: 1795
前10个节点:
节点信息已成功保存到 nodes_data.json
